In [1]:
!blastn -task blastn -query data/res/sRNA_DEG.fasta  -subject data/data/miRNAs/mature.fa -outfmt "6 qseqid sseqid qstart qend sstart send score length mismatch gaps gapopen nident pident qlen slen qcovs" > data/res/sRNA_DEG_mirbase.blast.csv

In [139]:
import pandas as pd
import numpy as np
data_dir = 'data'
lb = 'iwgsc_10_12_18'
blast_out = data_dir + '/res/sRNA_DEG_mirbase.blast.csv'

In [148]:
deg_file = data_dir + '/res/deg_data.csv'
df_deg = pd.read_csv(deg_file,sep="\t")
df_deg.loc[pd.isnull(df_deg['MajorRNA']), 'MajorRNA'] = df_deg['Gene']
print('Res len:',len(df_deg.index))
df_deg.head(3)

Res len: 19180


,Unnamed: 0,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1,UUGUACCAACUGUAAACUCGU,2848.856383,11.769840,0.889328,13.234532,5.544114e-40,3.126881e-35,2.069231,1.148383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,AGAUUCUGUUGAACUCGUUCUC,2065.048285,11.302687,0.872536,12.953837,2.234989e-38,6.302670e-34,1.034615,2.296765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,AAUGAACAUCACAAACAGCCU,1217.082821,10.128707,0.789553,12.828409,1.136739e-37,2.137069e-33,3.103846,1.148383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
df_blast = pd.read_csv(blast_out,sep="\t", header=None)
df_blast.columns = ['qseqid','sseqid','qstart','qend','sstart','send','score','length','mismatch','gaps','gapopen','nident','pident','qlen','slen','qcovs']
print('initial:',len(df_blast.index))

initial: 78432


In [150]:
df_blast = df_blast[df_blast.length >= df_blast.qlen - 4]
print('len -4:',len(df_blast.index))

len -4: 13263


In [151]:
df_res = pd.merge(df_deg, df_blast[['qseqid','sseqid']], left_on='Gene', right_on='qseqid', how='left')
df_res = df_res.drop(['qseqid'], axis=1)
df_res.rename(columns={'sseqid': 'miRBASE'}, inplace=True)
df_res = df_res.replace(np.nan, '', regex=True)
print(len(df_res.index))
df_res.head(2)

32015


,Unnamed: 0,Gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,miRBASE
0,1,UUGUACCAACUGUAAACUCGU,2848.856383,11.769840,0.889328,13.234532,5.544114e-40,3.126881e-35,2.069231,1.148383,...,,,,,,,,,,
1,2,AGAUUCUGUUGAACUCGUUCUC,2065.048285,11.302687,0.872536,12.953837,2.234989e-38,6.302670e-34,1.034615,2.296765,...,,,,,,,,,,


In [152]:
rules = {'MajorRNA':'first','log2FoldChange':'first','miRBASE':','.join}
df_res = df_res.groupby('Gene',as_index=False).agg(rules)

In [153]:
df_res.to_csv(data_dir + '/res/sRNA_DEG_mirbase.csv', sep='\t', index=None)